In [1]:
# import osmnx as ox
import pandas as pd
import geopandas as gpd
import json
from io import StringIO
import warnings
import sys
import os
warnings.filterwarnings("ignore")
sys.stderr = open(os.devnull, 'w')

# local crs
local_crs = 32636

In [2]:
from popframe.models.region import Region

region_model = Region.from_pickle('data/Ленинградская область.pickle')

In [3]:
from shapely.geometry import shape
geojson_data = {
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [29.982879431084967, 59.363554752593245],
                [29.982879431084967, 59.322083801173534],
                [30.109075699649765, 59.322083801173534],
                [30.109075699649765, 59.363554752593245],
                [29.982879431084967, 59.363554752593245]
            ]
        ]
    },
    "properties": {},
    "type": "Feature"
}

geom = shape(geojson_data['geometry'])

# Создаем GeoDataFrame
territory = gpd.GeoDataFrame([{'geometry': geom}], crs='EPSG:4326')

In [4]:
from popframe.method.territory_evaluation import TerritoryEvaluation
from popframe.models.region import Region
from shapely.geometry import shape

evaluation = TerritoryEvaluation(region=region_model)

geojson_data = {
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [29.982879431084967, 59.363554752593245],
                [29.982879431084967, 59.322083801173534],
                [30.109075699649765, 59.322083801173534],
                [30.109075699649765, 59.363554752593245],
                [29.982879431084967, 59.363554752593245]
            ]
        ]
    },
    "properties": {},
    "type": "Feature"
}

geom = shape(geojson_data['geometry'])

# Создаем GeoDataFrame
territories_gdf = gpd.GeoDataFrame([{'geometry': geom}], crs='EPSG:4326')

result = evaluation.evaluate_territory_location(territories=territories_gdf)
print(result)

[{'territory': 'None', 'score': 3, 'interpretation': 'территория находится внутри или непосредственной близости населенного пункта уровня Большое сельское поселение', 'closest_settlement': 'Сиверский', 'closest_settlement1': None, 'closest_settlement2': None}]
